# Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/')
%cd /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


# Load Data

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import numpy as np
import sys
from tqdm import tqdm

In [ ]:
from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128
default_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform = default_transform)

split_ratio = 0.8
total_size = len(trainset)
train_size = int(split_ratio * total_size)
valid_size = total_size - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

second_split = 0.01
total_size = len(train_dataset)
train_size = int(second_split * total_size)
discard_size = total_size - train_size
print(train_size)
train_dataset, discard_dataset = torch.utils.data.random_split(train_dataset, [train_size, discard_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform = default_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class_distribution = defaultdict(int)
for _, target in trainloader.dataset:
    class_name = classes[target]
    class_distribution[class_name] += 1
for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} samples")

Files already downloaded and verified
400
Files already downloaded and verified
horse: 40 samples
car: 36 samples
frog: 37 samples
plane: 46 samples
truck: 36 samples
deer: 45 samples
ship: 46 samples
cat: 39 samples
bird: 42 samples
dog: 33 samples


# ResNet50

In [ ]:
from torch.optim.lr_scheduler import StepLR

resnet50 = models.resnet50(pretrained=True)
num_classes = 10
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50 = resnet50.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=.005, momentum=0.9)

scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [ ]:
num_epochs = 60

for epoch in range(num_epochs):
  resnet50.train()
  running_loss = 0.0

  for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = resnet50(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  scheduler.step()

  print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader)}')

resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = resnet50(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')

Epoch 1/60, Loss: 2.346644937992096
Epoch 2/60, Loss: 2.00325208902359
Epoch 3/60, Loss: 1.6216922104358673
Epoch 4/60, Loss: 1.3288804590702057
Epoch 5/60, Loss: 0.9773386120796204
Epoch 6/60, Loss: 0.7135595083236694
Epoch 7/60, Loss: 0.6224089935421944
Epoch 8/60, Loss: 0.6934807971119881
Epoch 9/60, Loss: 0.4361528158187866
Epoch 10/60, Loss: 0.5411928743124008
Epoch 11/60, Loss: 0.674762949347496
Epoch 12/60, Loss: 0.5911361649632454
Epoch 13/60, Loss: 0.5848597437143326
Epoch 14/60, Loss: 0.43174684047698975
Epoch 15/60, Loss: 0.6033132486045361
Epoch 16/60, Loss: 0.6530306339263916
Epoch 17/60, Loss: 0.6777842491865158
Epoch 18/60, Loss: 0.6756907999515533
Epoch 19/60, Loss: 0.7345828786492348
Epoch 20/60, Loss: 0.9794175326824188
Epoch 21/60, Loss: 0.5823468081653118
Epoch 22/60, Loss: 0.47096099704504013
Epoch 23/60, Loss: 0.6773055754601955
Epoch 24/60, Loss: 0.321736391633749
Epoch 25/60, Loss: 0.25817407108843327
Epoch 26/60, Loss: 0.3876474555581808
Epoch 27/60, Loss: 0.23

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import models

learning_rates = [1e-2, 1e-3]
num_epochs_list = [30,40,50]

best_accuracy = 0.0
best_params = {}
best_model = None

for lr in learning_rates:
    for num_epochs in num_epochs_list:

        resnet50 = models.resnet50(pretrained=True)
        num_classes = 10
        resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
        resnet50 = resnet50.to(device)


        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(resnet50.parameters(), lr=lr, momentum=0.9)
        scheduler = StepLR(optimizer, step_size=20, gamma=0.1)


        for epoch in range(num_epochs):
            resnet50.train()
            running_loss = 0.0

            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            scheduler.step()


        resnet50.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = resnet50(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'lr': lr, 'num_epochs': num_epochs}
            best_model = resnet50

print(f'Best Hyperparameters: {best_params}')
print(f'Best Valid Accuracy: {100 * best_accuracy:.2f}%')

correct = 0
total = 0

with torch.no_grad():
  for inputs, labels in testloader:
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = best_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')


Best Hyperparameters: {'lr': 0.01, 'num_epochs': 50}
Best Valid Accuracy: 40.36%
Test Accuracy: 40.77%


# Save Model

In [ ]:
torch.save(resnet50.state_dict(), 'resnet.pth')

# Load Model

In [ ]:
state_dict = torch.load('resnet.pth', map_location=torch.device(device))

resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)
resnet50.to(device)

resnet50.load_state_dict(state_dict)

<All keys matched successfully>